In [3]:
# !pip install gradio ollama

In [4]:
import gradio as gr
import ollama
import re

In [50]:
# 1. 게임 상태 초기화
game_state = {
    "loop": 0,
    "memory": [],
    "correct_count": 0,
    "max_correct": 5,
    "game_over": False,
    "started": False  # ✅ 게임 시작 여부 체크 추가
}

def reset_game_state():
    game_state["loop"] = 0
    game_state["memory"].clear()
    game_state["correct_count"] = 0
    game_state["game_over"] = False
    game_state["started"] = False

In [55]:
# 2. 장면 생성 함수
def generate_scene(memory, loop_count):
    memory_text = "\n".join([f"- '{m[0]}' → {m[1]}" for m in memory[-5:]]) if memory else "기억 없음"
    system_prompt = """
당신은 영화 '엣지 오브 투모로우'의 빌 케이지 소령처럼 전장을 반복하는 병사입니다.
규칙:
- AI는 반드시 적의 현재 전쟁의 상황의 설명만 생성합니다.
- 선택지는 간결하게 표현되어야 하며, 그 결과에 대한 유불리 정보를 포함하지 않습니다.
- 선택지는 반드시 다음과 같은 3가지로 고정:
선택지는 다음과 같습니다:
1. 돌진한다
2. 후퇴한다
3. 지원을 요청한다
"""
    user_prompt = f"""[루프 {loop_count}회차]
당신은 이전 루프에서 다음과 같은 행동을 했고, 그 결과를 모두 기억합니다:
{memory_text}
당신은 지금 어떤 상황에 처해 있으며, 어떤 전략적 판단이 필요한지 알려주세요."""
    response = ollama.chat(
        model="EEVE-Korean-10.8B",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response['message']['content'].strip()

In [56]:
# 3. 선택지 추출 함수
def extract_choices_from_response(response):
    pattern = r"선택지는 다음과 같습니다:\s*1\.\s*(.*?)\s*2\.\s*(.*?)\s*3\.\s*(.*?)\s*$"
    match = re.search(pattern, response, re.DOTALL)
    if not match:
        return ["돌진한다", "후퇴한다", "지원을 요청한다"]
    return [match.group(1).strip(), match.group(2).strip(), match.group(3).strip()]

In [57]:
# 4. 이벤트 처리 + 엔딩 체크
import random  # ✅ 확률 처리용 모듈 추가

def apply_event(choice, response):
    if game_state["game_over"]:
        return "🎉 게임은 이미 종료되었습니다. 새로 시작하려면 재실행하세요."

    if any(word in response for word in ["죽", "죽음", "사망", "전사", "폭발", "체력", "파괴", "관통"]) or random.random() < 0.3:  # ✅ 30% 확률로 사망 처리 추가
        game_state["memory"].append((choice, "죽음"))
        game_state["loop"] = 1 if game_state["loop"] == 0 else game_state["loop"] + 1
        return f"☠️ 당신은 죽었습니다. 루프가 다시 시작됩니다."

    game_state["memory"].append((choice, "생존"))
    game_state["correct_count"] += 1
    if game_state["correct_count"] >= game_state["max_correct"]:
        game_state["game_over"] = True
        boss_prompt = f"""[루프 {game_state['loop']}회차 – 최종 기억 통합 완료]
당신은 지금까지 {game_state['correct_count']}번의 올바른 선택을 통해 모든 기억을 되찾았습니다.
이제 당신은 최종 보스와 마주하게 됩니다.

보스는 엄청난 병력과 압도적인 화력으로 당신을 압박합니다.
지금까지의 기억이 당신의 판단에 도움이 될 수 있습니다.

최종 전투의 전황을 설명해 주세요. (선택지는 고정되어 있습니다)
선택지는 다음과 같습니다:
1. 돌진한다
2. 후퇴한다
3. 지원을 요청한다
"""
        # langchain api로 gemini api -> 플래시 뭐시기
        boss_response = ollama.chat(
            model="EEVE-Korean-10.8B",
            messages=[{"role": "user", "content": boss_prompt}]
        )
        boss_description = boss_response['message']['content'].strip()
        boss_text = boss_description.split("선택지는 다음과 같습니다:")[0].strip()

        if random.random() < 0.4:
            game_state["started"] = False
            return (
                f"{boss_text}\n👑 당신은 최종 보스를 물리쳤습니다! 지옥 같은 전장을 끝냈습니다.\n🎉 게임 클리어!",
                gr.update(choices=[], interactive=False),
                gr.update(visible=False),
                gr.update(visible=True)
            )
        else:
            game_state["memory"].append(("보스전", "죽음"))
            game_state["loop"] += 1
            return (
                f"{boss_text}\n☠️ 최종 전투에서 전사했습니다. 다시 반복됩니다.",
                gr.update(choices=[], interactive=False),
                gr.update(visible=False),
                gr.update(visible=True)
            )\n{boss_response['message']['content'].strip()}"

    game_state["loop"] += 1
    return f"[루프 {game_state['loop']}회차 시작]\n✅ 생존했습니다! (누적 생존 {game_state['correct_count']}회)"  # ✅ 생존 시 기억 출력 제거


In [58]:
# 5. Gradio 인터페이스
with gr.Blocks() as demo:
    with gr.Row():
        output_box = gr.Textbox(label="🧠 전장의 상황", lines=16, interactive=False)
    with gr.Row():
        choice_radio = gr.Radio(choices=[], label="무엇을 하시겠습니까?", type="index", interactive=True)
    with gr.Row():
        start_btn = gr.Button("🎮 게임 시작")
        submit_btn = gr.Button("결정!", visible=False)
        init_btn = gr.Button("🔄 게임 다시 시작하기")

    def show_intro():
        reset_game_state()
        intro_text = """
당신은 전쟁터에서 깨어났습니다. 먼 곳에서 포성이 들리고,
먼지와 연기가 자욱한 하늘 아래에 처해 있습니다.
첫 임무를 앞두고 상황을 파악해야 합니다.
"""
        return intro_text, gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

    def on_start():
        game_state["started"] = True
        first_response = generate_scene([], 0)
        initial_choices = extract_choices_from_response(first_response)
        description = first_response.split("선택지는 다음과 같습니다:")[0].strip()
        return description, gr.update(choices=initial_choices, value=None, interactive=True), gr.update(visible=True), gr.update(visible=False)

    def on_decide(choice):
        response = generate_scene(game_state["memory"], game_state["loop"])
        choices = extract_choices_from_response(response)
        result = apply_event(choices[choice], response)
        description = response.split("선택지는 다음과 같습니다:")[0].strip()

        if game_state["game_over"]:
            return f"{description}\n{result}", gr.update(choices=["🎉 게임 종료됨"], interactive=False), gr.update(visible=False), gr.update(visible=True)

        if "☠️ 당신은 죽었습니다" in result:
            return result, gr.update(choices=[], interactive=False), gr.update(visible=False), gr.update(visible=True)

        return f"{description}\n{result}", gr.update(choices=choices, value=None), gr.update(visible=True), gr.update(visible=False)

    start_btn.click(on_start, outputs=[output_box, choice_radio, submit_btn, start_btn])
    submit_btn.click(on_decide, inputs=choice_radio, outputs=[output_box, choice_radio, submit_btn, start_btn])
    init_btn.click(show_intro, outputs=[output_box, start_btn, submit_btn, init_btn])
    
    demo.load(show_intro, outputs=[output_box, start_btn, submit_btn, init_btn])
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7874
* Running on public URL: https://2a57fd1de00696cb3b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
